In [ ]:
!pip install rank-bm25 sentence-transformers transformers torch tqdm

In [3]:
import re
import math
import torch
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
question = "Which are the top five best selling products? Please state the full name of them."
evidence = "Description of products contains full name"
schema = r"""
【DB_ID】 debit_card_specializing
【Schema】
# Table: customers
[
(CustomerID:INTEGER, Primary Key, Examples: [3, 5, 6]),
(Segment:TEXT, Examples: [SME, LAM, KAM]),
(Currency:TEXT, Examples: [EUR, CZK])
]
# Table: yearmonth
[
(CustomerID:INTEGER, Primary Key, Examples: [39, 63, 172]),
(Date:TEXT, Primary Key, Examples: [201112, 201201, 201202]),
(Consumption:REAL, Examples: [528.3, 1598.28, 1931.36])
]
# Table: products
[
(ProductID:INTEGER, Primary Key, Examples: [1, 2, 3]),
(Description:TEXT, Examples: [Rucní zadání, Nafta, Special])
]
# Table: gasstations
[
(GasStationID:INTEGER, Primary Key, Examples: [44, 45, 46]),
(ChainID:INTEGER, Examples: [13, 6, 23]),
(Country:TEXT, Examples: [CZE, SVK]),
(Segment:TEXT, Examples: [Value for money, Premium, Other])
]
# Table: transactions_1k
[
(TransactionID:INTEGER, Primary Key, Examples: [1, 2, 3]),
(Date:DATE, Examples: [2012-08-24]),
(Time:TEXT, Examples: [09:41:00, 10:03:00, 13:53:00]),
(CustomerID:INTEGER, Examples: [31543, 46707, 7654]),
(CardID:INTEGER, Examples: [486621, 550134, 684220]),
(GasStationID:INTEGER, Examples: [3704, 656, 741]),
(ProductID:INTEGER, Examples: [2, 23, 5]),
(Amount:INTEGER, Examples: [28, 18, 1]),
(Price:REAL, Examples: [672.64, 430.72, 121.99])
]
【Foreign keys】
yearmonth.CustomerID=customers.CustomerID
"""

In [ ]:
question = "The average unemployment ratio of 1995 and 1996, which one has higher percentage?"
evidence = "A12 refers to unemploymant rate 1995; A13 refers to unemploymant rate 1996"
schema = r"""
【DB_ID】 financial
【Schema】
# Table: client
[
(client_id:INTEGER, Primary Key, Examples: [1, 2, 3]),
(gender:TEXT, Examples: [F, M]),
(birth_date:DATE, Examples: [1970-12-13]),
(district_id:INTEGER, Examples: [18, 1, 5])
]
# Table: order
[
(order_id:INTEGER, Primary Key, Examples: [29401, 29402, 29403]),
(account_id:INTEGER, Examples: [1, 2, 3]),
(bank_to:TEXT, Examples: [YZ, ST, QR]),
(account_to:INTEGER, Examples: [87144583, 89597016, 13943797]),
(amount:REAL, Examples: [2452.0, 3372.7, 7266.0]),
(k_symbol:TEXT, Examples: [SIPO, UVER, POJISTNE])
]
# Table: account
[
(account_id:INTEGER, Primary Key, Examples: [1, 2, 3]),
(district_id:INTEGER, Examples: [18, 1, 5]),
(frequency:TEXT, Examples: [POPLATEK MESICNE, POPLATEK TYDNE, POPLATEK PO OBRATU]),
(date:DATE, Examples: [1995-03-24])
]
# Table: card
[
(card_id:INTEGER, Primary Key, Examples: [1, 2, 3]),
(disp_id:INTEGER, Examples: [9, 19, 41]),
(type:TEXT, Examples: [gold, classic, junior]),
(issued:DATE, Examples: [1998-10-16])
]
# Table: district
[
(district_id:INTEGER, Primary Key, Examples: [1, 2, 3]),
(A2:TEXT, Examples: [Hl.m. Praha, Benesov, Beroun]),
(A3:TEXT, Examples: [Prague, central Bohemia, south Bohemia]),
(A4:TEXT, Examples: [1204953, 88884, 75232]),
(A5:TEXT, Examples: [0, 80, 55]),
(A6:TEXT, Examples: [0, 26, 29]),
(A7:TEXT, Examples: [0, 6, 4]),
(A8:INTEGER, Examples: [1, 2, 3]),
(A9:INTEGER, Examples: [1, 5, 6]),
(A10:REAL, Examples: [100.0, 46.7, 41.7]),
(A11:INTEGER, Examples: [12541, 8507, 8980]),
(A12:REAL, Examples: [0.2, 1.6, 1.9]),
(A13:REAL, Examples: [0.43, 1.85, 2.21]),
(A14:INTEGER, Examples: [167, 132, 111]),
(A15:INTEGER, Examples: [85677, 2159, 2824]),
(A16:INTEGER, Examples: [99107, 2674, 2813])
]
# Table: disp
[
(disp_id:INTEGER, Primary Key, Examples: [1, 2, 3]),
(client_id:INTEGER, Examples: [1, 2, 3]),
(account_id:INTEGER, Examples: [1, 2, 3]),
(type:TEXT, Examples: [OWNER, DISPONENT])
]
# Table: trans
[
(trans_id:INTEGER, Primary Key, Examples: [1, 5, 6]),
(account_id:INTEGER, Examples: [1, 2, 3]),
(date:DATE, Examples: [1995-03-24]),
(type:TEXT, Examples: [PRIJEM, VYDAJ, VYBER]),
(operation:TEXT, Examples: [VKLAD, PREVOD Z UCTU, PREVOD NA UCET]),
(amount:INTEGER, Examples: [1000, 3679, 2452]),
(balance:INTEGER, Examples: [1000, 4679, 20977]),
(k_symbol:TEXT, Examples: [SIPO, SLUZBY, UVER]),
(bank:TEXT, Examples: [AB, YZ, ST]),
(account:INTEGER, Examples: [41403269, 87144583, 66487163])
]
# Table: loan
[
(loan_id:INTEGER, Primary Key, Examples: [4959, 4961, 4962]),
(account_id:INTEGER, Examples: [2, 19, 25]),
(date:DATE, Examples: [1994-01-05]),
(amount:INTEGER, Examples: [80952, 30276, 318480]),
(duration:INTEGER, Examples: [24, 12, 60]),
(payments:REAL, Examples: [3373.0, 2523.0, 5308.0]),
(status:TEXT, Examples: [A, B, D])
]
【Foreign keys】
client.district_id=district.district_id
order.account_id=account.account_id
account.district_id=district.district_id
card.disp_id=disp.disp_id
disp.account_id=account.account_id
disp.client_id=client.client_id
trans.account_id=account.account_id
loan.account_id=account.account_id
"""

In [20]:
question = "What is the highest eligible free rate for K-12 students in the schools in Alameda County?"
evidence = "Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`"
schema = r"""
【DB_ID】 california_schools
【Schema】
# Table: schools
[
(CDSCode:TEXT, Primary Key, Examples: [01100170000000, 01100170109835, 01100170112607]),
(NCESDist:TEXT, Examples: [0691051, 0600002, 0600003]),
(NCESSchool:TEXT, Examples: [10546, 10947, 12283]),
(StatusType:TEXT, Examples: [Active, Closed, Merged]),
(County:TEXT, Examples: [Alameda, Alpine, Amador]),
(District:TEXT),
(School:TEXT, Examples: [FAME Public Charter]),
(Street:TEXT, Examples: [313 West Winton Avenue]),
(StreetAbr:TEXT, Examples: [313 West Winton Ave.]),
(City:TEXT, Examples: [Hayward, Newark, Oakland]),
(Zip:TEXT, Examples: [94544-1136, 94560-5359, 94612-3355]),
(State:TEXT, Examples: [CA]),
(MailStreet:TEXT, Examples: [313 West Winton Avenue]),
(MailStrAbr:TEXT, Examples: [313 West Winton Ave.]),
(MailCity:TEXT, Examples: [Hayward, Newark, Oakland]),
(MailZip:TEXT, Examples: [94544-1136, 94560-5359, 94612]),
(MailState:TEXT, Examples: [CA]),
(Phone:TEXT, Examples: [(510) 887-0152, (510) 596-8901, (510) 686-4131]),
(Ext:TEXT, Examples: [130, 1240, 1200]),
(Website:TEXT, Examples: [www.acoe.org]),
(OpenDate:DATE, Examples: [2005-08-29]),
(ClosedDate:DATE, Examples: [2015-07-31]),
(Charter:INTEGER, Examples: [1, 0]),
(CharterNum:TEXT, Examples: [0728, 0811, 1049]),
(FundingType:TEXT, Examples: [Directly funded]),
(DOC:TEXT, Examples: [00, 31, 34]),
(DOCType:TEXT, Examples: [County Office of Education (COE)]),
(SOC:TEXT, Examples: [65, 66, 60]),
(SOCType:TEXT, Examples: [K-12 Schools (Public)]),
(EdOpsCode:TEXT, Examples: [TRAD, JUV, COMM]),
(EdOpsName:TEXT, Examples: [Traditional]),
(EILCode:TEXT, Examples: [ELEMHIGH, HS, ELEM]),
(EILName:TEXT, Examples: [Elementary-High Combination]),
(GSoffered:TEXT, Examples: [K-12, 9-12, K-8]),
(GSserved:TEXT, Examples: [K-12, 9-12, K-7]),
(Virtual:TEXT, Examples: [P, N, F]),
(Magnet:INTEGER, Examples: [0, 1]),
(Latitude:REAL, Examples: [37.658212, 37.521436, 37.80452]),
(Longitude:REAL, Examples: [-122.09713, -121.99391, -122.26815]),
(AdmFName1:TEXT, Examples: [L Karen, Laura, Clifford]),
(AdmLName1:TEXT, Examples: [Monroe, Robell, Thompson]),
(AdmEmail1:TEXT),
(AdmFName2:TEXT, Examples: [Sau-Lim (Lance), Jennifer, Annalisa]),
(AdmLName2:TEXT, Examples: [Tsang, Koelling, Moore]),
(AdmEmail2:TEXT),
(AdmFName3:TEXT, Examples: [Drew, Irma, Vickie]),
(AdmLName3:TEXT, Examples: [Sarratore, Munoz, Chang]),
(AdmEmail3:TEXT),
(LastUpdate:DATE, Examples: [2015-06-23])
]
# Table: satscores
[
(cds:TEXT, Primary Key, Examples: [10101080000000, 10101080109991, 10101080111682]),
(rtype:TEXT, Examples: [D, S]),
(sname:TEXT, Examples: [FAME Public Charter]),
(dname:TEXT, Examples: [Alameda County Office of Education]),
(cname:TEXT, Examples: [Alameda, Amador, Butte]),
(enroll12:INTEGER, Examples: [398, 62, 75]),
(NumTstTakr:INTEGER, Examples: [88, 17, 71]),
(AvgScrRead:INTEGER, Examples: [418, 503, 397]),
(AvgScrMath:INTEGER, Examples: [418, 546, 387]),
(AvgScrWrite:INTEGER, Examples: [417, 505, 395]),
(NumGE1500:INTEGER, Examples: [14, 9, 5])
]
# Table: frpm
[
(CDSCode:TEXT, Primary Key, Examples: [01100170109835, 01100170112607, 01100170118489]),
(Academic Year:TEXT, Examples: [2014-2015]),
(County Code:TEXT, Examples: [01, 02, 03]),
(District Code:INTEGER, Examples: [10017, 31609, 31617]),
(School Code:TEXT, Examples: [0109835, 0112607, 0118489]),
(County Name:TEXT, Examples: [Alameda, Alpine, Amador]),
(District Name:TEXT),
(School Name:TEXT, Examples: [FAME Public Charter]),
(District Type:TEXT, Examples: [County Office of Education (COE)]),
(School Type:TEXT, Examples: [K-12 Schools (Public)]),
(Educational Option Type:TEXT, Examples: [Traditional]),
(NSLP Provision Status:TEXT, Examples: [Breakfast Provision 2]),
(Charter School (Y/N):INTEGER, Examples: [1, 0]),
(Charter School Number:TEXT, Examples: [0728, 0811, 1049]),
(Charter Funding Type:TEXT, Examples: [Directly funded]),
(IRC:INTEGER, Examples: [1, 0]),
(Low Grade:TEXT, Examples: [K, 9, 1]),
(High Grade:TEXT, Examples: [12, 8, 5]),
(Enrollment (K-12):REAL, Examples: [1087.0, 395.0, 244.0]),
(Free Meal Count (K-12):REAL, Examples: [565.0, 186.0, 134.0]),
(Percent (%) Eligible Free (K-12):REAL, Examples: [0.519779208831647, 0.470886075949367, 0.549180327868853]),
(FRPM Count (K-12):REAL, Examples: [715.0, 186.0, 175.0]),
(Percent (%) Eligible FRPM (K-12):REAL, Examples: [0.657773689052438, 0.470886075949367, 0.717213114754098]),
(Enrollment (Ages 5-17):REAL, Examples: [1070.0, 376.0, 230.0]),
(Free Meal Count (Ages 5-17):REAL, Examples: [553.0, 182.0, 128.0]),
(Percent (%) Eligible Free (Ages 5-17):REAL, Examples: [0.516822429906542, 0.484042553191489, 0.556521739130435]),
(FRPM Count (Ages 5-17):REAL, Examples: [702.0, 182.0, 168.0]),
(Percent (%) Eligible FRPM (Ages 5-17):REAL, Examples: [0.65607476635514, 0.484042553191489, 0.730434782608696]),
(2013-14 CALPADS Fall 1 Certification Status:INTEGER, Examples: [1])
]
【Foreign keys】
satscores.cds=schools.CDSCode
frpm.CDSCode=schools.CDSCode
"""

In [21]:
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional, Any

@dataclass
class Column:
    name: str
    col_type: Optional[str] = None
    description: Optional[str] = None
    examples: List[str] = field(default_factory=list)

@dataclass
class Table:
    name: str
    columns: List[Column]

@dataclass
class ForeignKey:
    src_table: str
    src_col: str
    dst_table: str
    dst_col: str

@dataclass
class DBSchema:
    db_id: Optional[str]
    tables: List[Table]
    foreign_keys: List[ForeignKey] = field(default_factory=list)

@dataclass
class Doc:
    doc_id: str
    table: str
    level: str
    text: str
    column: Optional[str] = None

In [22]:
def parse_schema_text(schema_text: str) -> DBSchema:
    """Lightweight line-based parser for the provided schema format.

    Assumptions:
    - Each column is on its own line like: (Name:TYPE, Primary Key, Examples: [a, b, c])
    - The first ':' splits column name and the rest.
    - TYPE ends at the next ',' or ')' (so extra flags like 'Primary Key' are safely ignored).
    - Examples are on the same line; we grab text between '[' and ']' and split by ','.
      Elements may contain parentheses; that's fine because we split on commas only.
    """
    # DB id
    m = re.compile(r"【DB_ID】\s*(?P<dbid>.+)").search(schema_text)
    db_id = m.group("dbid").strip() if m else None

    tables: List[Table] = []
    foreign_keys: List[ForeignKey] = []

    # Split into table blocks
    parts = re.split(r"(?=#\s*Table:)", schema_text)
    for block in parts:
        mh = re.compile(r"#\s*Table:\s*(?P<name>\w+)").search(block)
        if not mh:
            continue
        tname = mh.group("name").strip()

        cols: List[Column] = []
        in_list = False
        for raw in block.splitlines():
            s = raw.strip().rstrip(",")  # tolerate trailing commas
            if s == "[":
                in_list = True
                continue
            if s == "]":
                in_list = False
                continue
            if not in_list:
                continue
            if not (s.startswith("(") and s.endswith(")")):
                continue

            # Strip outer parentheses
            body = s[1:-1].strip()
            if ":" not in body:
                continue

            # name : rest
            name, rest = body.split(":", 1)
            name = name.strip()

            # TYPE is up to next ',' (or end)
            rest = rest.strip()
            comma = rest.find(",")
            if comma >= 0:
                typ, tail = rest[:comma].strip(), rest[comma+1:].strip()
            else:
                typ, tail = rest, ""

            # Examples (same line)
            examples: List[str] = []
            if "Examples" in tail:
                lb, rb = tail.find("["), tail.rfind("]")
                if lb != -1 and rb != -1 and rb > lb:
                    content = tail[lb+1:rb]
                    examples = [e.strip() for e in content.split(",") if e.strip()]

            cols.append(Column(name=name, col_type=typ, examples=examples[:3]))

        tables.append(Table(name=tname, columns=cols))

    # Foreign keys (keep your original pattern)
    fk_block = re.search(r"【Foreign keys】(.+)$", schema_text, re.DOTALL)
    if fk_block:
        for line in fk_block.group(1).strip().splitlines():
            m = re.compile(r"(\w+)\.(\w+)=(\w+)\.(\w+)").search(line.strip())
            if m:
                foreign_keys.append(ForeignKey(
                    src_table=m.group(1), src_col=m.group(2),
                    dst_table=m.group(3), dst_col=m.group(4)
                ))

    return DBSchema(db_id=db_id, tables=tables, foreign_keys=foreign_keys)

schema_obj = parse_schema_text(schema_text=schema)
schema_obj.tables[0].columns[:]

[Column(name='CDSCode', col_type='TEXT', description=None, examples=['01100170000000', '01100170109835', '01100170112607']),
 Column(name='NCESDist', col_type='TEXT', description=None, examples=['0691051', '0600002', '0600003']),
 Column(name='NCESSchool', col_type='TEXT', description=None, examples=['10546', '10947', '12283']),
 Column(name='StatusType', col_type='TEXT', description=None, examples=['Active', 'Closed', 'Merged']),
 Column(name='County', col_type='TEXT', description=None, examples=['Alameda', 'Alpine', 'Amador']),
 Column(name='District', col_type='TEXT', description=None, examples=[]),
 Column(name='School', col_type='TEXT', description=None, examples=['FAME Public Charter']),
 Column(name='Street', col_type='TEXT', description=None, examples=['313 West Winton Avenue']),
 Column(name='StreetAbr', col_type='TEXT', description=None, examples=['313 West Winton Ave.']),
 Column(name='City', col_type='TEXT', description=None, examples=['Hayward', 'Newark', 'Oakland']),
 Col

In [25]:
def build_docs(schema: DBSchema, include_examples: bool = True) -> List[Doc]:
    """construct documents"""
    
    docs: List[Doc] = []
    for t in schema.tables:
        table_desc = f"Table {t.name}. Columns: " + ", ".join([c.name for c in t.columns][:20])
        docs.append(Doc(doc_id=t.name, table=t.name, level="table", text=table_desc))
        for c in t.columns:
            ex_part = f" Examples: {', '.join(c.examples)}" if include_examples and c.examples else ""
            txt = f"Column {c.name} in table {t.name}. Type: {c.col_type}.{ex_part}"
            docs.append(Doc(doc_id=f"{t.name}|{c.name}", table=t.name, level="column", column=c.name, text=txt))
    return docs

docs = build_docs(schema=schema_obj, include_examples=True)
id2doc = {d.doc_id: d for d in docs}
id2doc.keys(), docs[:10]

(dict_keys(['schools', 'schools|CDSCode', 'schools|NCESDist', 'schools|NCESSchool', 'schools|StatusType', 'schools|County', 'schools|District', 'schools|School', 'schools|Street', 'schools|StreetAbr', 'schools|City', 'schools|Zip', 'schools|State', 'schools|MailStreet', 'schools|MailStrAbr', 'schools|MailCity', 'schools|MailZip', 'schools|MailState', 'schools|Phone', 'schools|Ext', 'schools|Website', 'schools|OpenDate', 'schools|ClosedDate', 'schools|Charter', 'schools|CharterNum', 'schools|FundingType', 'schools|DOC', 'schools|DOCType', 'schools|SOC', 'schools|SOCType', 'schools|EdOpsCode', 'schools|EdOpsName', 'schools|EILCode', 'schools|EILName', 'schools|GSoffered', 'schools|GSserved', 'schools|Virtual', 'schools|Magnet', 'schools|Latitude', 'schools|Longitude', 'schools|AdmFName1', 'schools|AdmLName1', 'schools|AdmEmail1', 'schools|AdmFName2', 'schools|AdmLName2', 'schools|AdmEmail2', 'schools|AdmFName3', 'schools|AdmLName3', 'schools|AdmEmail3', 'schools|LastUpdate', 'satscores',

In [26]:
query = question.strip()
query = query + "\n" + evidence.strip() if evidence else query
query

'What is the highest eligible free rate for K-12 students in the schools in Alameda County?\nEligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`'

In [39]:
def bm25_scores(query: str, docs: List[Doc]) -> Dict[str, float]:
    """sparse model bm25 api"""
    
    tokenized_corpus = [[w.lower() for w in re.findall(r"\w+", d.text)] for d in docs]
    tokenized_query = [w.lower() for w in re.findall(r"\w+", query)]
    bm25 = BM25Okapi(tokenized_corpus)
    scores = bm25.get_scores(tokenized_query)
    return {docs[i].doc_id: float(scores[i]) for i in range(len(docs))}

bm25_sd: Dict[str, float] = bm25_scores(query=query, docs=docs)
sorted_bm25_sd = sorted(bm25_sd.items(), key=lambda kv: kv[1], reverse=True)
sorted_bm25_sd

[('frpm|Free Meal Count (K-12)', 26.6983440578652),
 ('frpm|Percent (%) Eligible Free (K-12)', 26.655900951748002),
 ('schools|GSoffered', 24.217672507528153),
 ('schools|GSserved', 24.217672507528153),
 ('frpm', 20.42235842304528),
 ('frpm|Percent (%) Eligible FRPM (K-12)', 19.68175461976518),
 ('schools|SOCType', 19.190718349505556),
 ('frpm|Enrollment (K-12)', 18.505483458719308),
 ('frpm|School Type', 18.14376083473703),
 ('frpm|FRPM Count (K-12)', 17.52442400665118),
 ('frpm|Free Meal Count (Ages 5-17)', 13.143121759592177),
 ('frpm|Percent (%) Eligible Free (Ages 5-17)', 13.101910181785357),
 ('frpm|Low Grade', 9.86325877153857),
 ('frpm|High Grade', 9.86325877153857),
 ('schools|County', 8.475847653212288),
 ('frpm|County Name', 7.245115224305003),
 ('satscores|dname', 6.990577062147648),
 ('frpm|Percent (%) Eligible FRPM (Ages 5-17)', 6.330125538997999),
 ('satscores|cname', 5.078819129559845),
 ('schools|DOCType', 4.937771391855742),
 ('frpm|Enrollment (Ages 5-17)', 4.36753201

In [ ]:
dense_bge = SentenceTransformer("/models/bge-m3")
dense_bge = dense_bge.to("cuda")

In [45]:
def dense_scores(query: str, docs: List[Doc], dense_bge: SentenceTransformer) -> Dict[str, float]:
    qv = dense_bge.encode([query], normalize_embeddings=True)
    dv = dense_bge.encode([d.text for d in docs], normalize_embeddings=True)
    sims = (dv @ qv[0])
    return {docs[i].doc_id: float(sims[i]) for i in range(len(docs))}

dense_sd: Dict[str, float] = dense_scores(query=query, docs=docs, dense_bge=dense_bge)
sorted_dense_sd = sorted(dense_sd.items(), key=lambda kv: kv[1], reverse=True)
sorted_dense_sd

[('frpm|Free Meal Count (K-12)', 0.6616315841674805),
 ('frpm|Percent (%) Eligible Free (K-12)', 0.6611534357070923),
 ('frpm', 0.6374523043632507),
 ('frpm|Free Meal Count (Ages 5-17)', 0.6203795075416565),
 ('frpm|Percent (%) Eligible FRPM (K-12)', 0.61309814453125),
 ('frpm|Percent (%) Eligible Free (Ages 5-17)', 0.6108018159866333),
 ('schools|GSoffered', 0.5750449299812317),
 ('frpm|Percent (%) Eligible FRPM (Ages 5-17)', 0.5706623792648315),
 ('frpm|Enrollment (K-12)', 0.5627457499504089),
 ('frpm|FRPM Count (K-12)', 0.5536397099494934),
 ('schools|GSserved', 0.551501989364624),
 ('schools|County', 0.54672771692276),
 ('frpm|School Type', 0.5464367866516113),
 ('frpm|Enrollment (Ages 5-17)', 0.5314897298812866),
 ('schools|EILCode', 0.530124843120575),
 ('frpm|FRPM Count (Ages 5-17)', 0.5184066891670227),
 ('schools|EILName', 0.5120658278465271),
 ('schools|Ext', 0.5113561749458313),
 ('schools|Phone', 0.5108435750007629),
 ('frpm|District Type', 0.5046148896217346),
 ('frpm|Coun

In [51]:
def rrf_fuse(*score_dicts: Dict[str, float], k: int = 60) -> Dict[str, float]:
    """Reciprocal Rank Fusion"""

    ranks_list: List[Dict[str, int]] = []
    for sd in score_dicts:
        sorted_ids = [i for i, _ in sorted(sd.items(), key=lambda kv: kv[1], reverse=True)]
        ranks_list.append({doc_id: r for r, doc_id in enumerate(sorted_ids, start=1)})

    all_ids = set()
    for sd in score_dicts:
        all_ids.update(sd.keys())

    fused: Dict[str, float] = {}
    for doc_id in all_ids:
        s = 0.0
        for ranks in ranks_list:
            if doc_id in ranks:
                s += 1.0 / (k + ranks[doc_id])
        fused[doc_id] = s
    return fused

fused = rrf_fuse(bm25_sd, dense_sd, k=60)
fused

{'schools|MailState': 0.020915032679738564,
 'satscores|cds': 0.01418546365914787,
 'satscores|dname': 0.024350649350649352,
 'frpm|FRPM Count (Ages 5-17)': 0.02506265664160401,
 'frpm|School Type': 0.02819138376017471,
 'schools|ClosedDate': 0.01739782527184102,
 'schools|School': 0.02133696515718988,
 'schools|AdmEmail2': 0.021595262974573322,
 'schools|GSoffered': 0.030798389007344232,
 'satscores|NumGE1500': 0.014039408866995073,
 'frpm|County Name': 0.025503573749187783,
 'frpm|Enrollment (Ages 5-17)': 0.025859192525859193,
 'schools|DOCType': 0.02399425287356322,
 'frpm|Percent (%) Eligible FRPM (Ages 5-17)': 0.027526395173453996,
 'schools|Website': 0.02046142603612068,
 'schools|NCESSchool': 0.021178843567906766,
 'schools|EdOpsCode': 0.017174311926605505,
 'schools|MailStrAbr': 0.016960114983830397,
 'satscores|NumTstTakr': 0.01393188854489164,
 'schools|AdmLName3': 0.015470004723665564,
 'schools|Magnet': 0.019729206963249514,
 'frpm|FRPM Count (K-12)': 0.02857142857142857,
 

In [52]:
cand_ids = [d.doc_id for d in sorted(docs, key=lambda d: fused.get(d.doc_id, 0.0), reverse=True)]
cand_ids = [cid for cid in cand_ids if id2doc[cid].level == "column"] + [cid for cid in cand_ids if id2doc[cid].level == "table"]
cand_ids = cand_ids[:50]
cand_ids

['frpm|Free Meal Count (K-12)',
 'frpm|Percent (%) Eligible Free (K-12)',
 'schools|GSoffered',
 'frpm|Percent (%) Eligible FRPM (K-12)',
 'schools|GSserved',
 'frpm|Free Meal Count (Ages 5-17)',
 'frpm|Enrollment (K-12)',
 'frpm|Percent (%) Eligible Free (Ages 5-17)',
 'frpm|FRPM Count (K-12)',
 'frpm|School Type',
 'frpm|Percent (%) Eligible FRPM (Ages 5-17)',
 'schools|County',
 'schools|SOCType',
 'frpm|Enrollment (Ages 5-17)',
 'frpm|County Name',
 'frpm|FRPM Count (Ages 5-17)',
 'frpm|District Type',
 'satscores|dname',
 'frpm|High Grade',
 'schools|DOCType',
 'schools|District',
 'frpm|Low Grade',
 'schools|Ext',
 'schools|EILCode',
 'schools|EILName',
 'schools|FundingType',
 'schools|State',
 'schools|AdmEmail2',
 'schools|School',
 'schools|NCESSchool',
 'schools|AdmEmail1',
 'schools|CharterNum',
 'satscores|cname',
 'schools|MailState',
 'frpm|County Code',
 'schools|Charter',
 'schools|Phone',
 'schools|Website',
 'schools|CDSCode',
 'schools|StatusType',
 'schools|Magnet'

In [62]:
class CrossEncoderReranker:
    def __init__(self, model_name: str = "BAAI/bge-reranker-base", device: Optional[str] = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        if device and hasattr(self.model, "to"):
            self.model = self.model.to(device)

    def score(self, query: str, texts: List[str], batch_size: int = 16) -> List[float]:
        self.model.eval()
        scores: List[float] = []
        with torch.no_grad():
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i+batch_size]
                inputs = self.tokenizer([query] * len(batch), batch, padding=True, truncation=True, return_tensors="pt")
                if hasattr(self.model, "device"):
                    inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
                logits = self.model(**inputs).logits.squeeze(-1)
                logits = logits.unsqueeze(0) if logits.ndim == 0 else logits
                batch_scores = logits.detach().cpu().tolist()
                batch_scores = [batch_scores] if isinstance(batch_scores, float) else batch_scores
                scores.extend([float(s) for s in batch_scores])
        return scores

# reranker = CrossEncoderReranker("/models/bge-reranker-v2-m3", "cuda")
reranker = CrossEncoderReranker("/models/bge-reranker-base", "cuda:2")
rerank_sd: Dict[str, float] = {}

In [64]:
texts = [id2doc[cid].text for cid in cand_ids]
scores = reranker.score(query, texts)
rerank_sd = {cand_ids[i]: float(scores[i]) for i in range(len(cand_ids))}
sorted_rerank_sd = sorted(rerank_sd.items(), key=lambda kv: kv[1], reverse=True)
sorted_rerank_sd

[('frpm|Free Meal Count (K-12)', 1.908007025718689),
 ('frpm|Percent (%) Eligible Free (K-12)', -0.2728872001171112),
 ('frpm|Enrollment (K-12)', -0.2964431345462799),
 ('frpm|Free Meal Count (Ages 5-17)', -2.9901041984558105),
 ('frpm|FRPM Count (K-12)', -3.2765893936157227),
 ('schools|GSserved', -3.735017776489258),
 ('frpm|Percent (%) Eligible FRPM (K-12)', -4.0866265296936035),
 ('schools|GSoffered', -4.193419933319092),
 ('schools|SOCType', -4.931002140045166),
 ('satscores|dname', -4.974353313446045),
 ('frpm|School Type', -5.034274578094482),
 ('schools|County', -5.492921352386475),
 ('frpm|Percent (%) Eligible Free (Ages 5-17)', -6.00980806350708),
 ('frpm|County Name', -6.794225215911865),
 ('frpm|Enrollment (Ages 5-17)', -6.858329772949219),
 ('schools|Website', -7.272578239440918),
 ('schools|EILCode', -7.466245174407959),
 ('schools|AdmEmail1', -7.531799793243408),
 ('schools|CDSCode', -7.630288600921631),
 ('schools|DOCType', -7.663693904876709),
 ('schools|AdmEmail2', -7

In [76]:
def prune_by_scores(
    docs: List[Doc],
    fused_scores: Dict[str, float],
    rerank_scores: Optional[Dict[str, float]] = None,
    topk_columns: int = 40,
    per_table_col_cap: int = 8,
) -> Tuple[Dict[str, List[str]], Dict[str, float]]:
    """return {table: [kept_columns]} and final score dict"""
    
    col_docs = [d for d in docs if d.level == "column"]
    final_scores: Dict[str, float] = {}
    
    for d in col_docs:
        base = fused_scores.get(d.doc_id, 0.0)
        rr = (rerank_scores or {}).get(d.doc_id, None)
        final_scores[d.doc_id] = ((float(rr) + float(base)) / 2.) if rr is not None else float(base) / 2.

    top_cols = sorted(col_docs, key=lambda d: final_scores.get(d.doc_id, 0.0), reverse=True)[:topk_columns]

    kept: Dict[str, List[str]] = {}
    for doc in top_cols:
        kept.setdefault(doc.table, [])
        if len(kept[doc.table]) < per_table_col_cap and doc.column:
            kept[doc.table].append(doc.column)

    return kept, final_scores

In [77]:
kept_cols, final_scores = prune_by_scores(
    docs=docs,
    fused_scores=fused,
    rerank_scores=rerank_sd,
    topk_columns=20,
    per_table_col_cap=10,
)

kept_cols, final_scores

({'frpm': ['Free Meal Count (K-12)',
   'District Code',
   'School Name',
   'Charter School (Y/N)',
   'Educational Option Type'],
  'schools': ['MailZip',
   'ClosedDate',
   'Virtual',
   'OpenDate',
   'EdOpsCode',
   'AdmFName1',
   'AdmLName1',
   'AdmLName2',
   'MailStrAbr',
   'SOC']},
 {'schools|CDSCode': -3.8050267736340397,
  'schools|NCESDist': -4.492028412833491,
  'schools|NCESSchool': -4.639622866118879,
  'schools|StatusType': -5.015413180999516,
  'schools|County': -2.7328495650821263,
  'schools|District': -4.024152278222033,
  'schools|School': -4.218306813441913,
  'schools|Street': 0.00796863141917531,
  'schools|StreetAbr': 0.008018474565398679,
  'schools|City': -4.571449748008482,
  'schools|Zip': 0.008385964912280702,
  'schools|State': -4.461721708815454,
  'schools|MailStreet': 0.008042635658914728,
  'schools|MailStrAbr': 0.008480057491915199,
  'schools|MailCity': -4.429316179134848,
  'schools|MailZip': 0.008782021782481976,
  'schools|MailState': -4.662

In [78]:
def prune_schema_min(schema: DBSchema, kept_cols: Dict[str, List[str]]) -> DBSchema:
    """
    Minimal pruning: keep only the tables present in `kept_cols` and only the listed columns.
    - Preserves original table/column order.
    - Silently ignores unknown tables/columns.
    - Keeps only FKs whose two endpoints both survive.
    """
    kept_tables = set(kept_cols.keys())

    new_tables: List[Table] = []
    for t in schema.tables:
        if t.name not in kept_tables:
            continue
        want = set(kept_cols[t.name])
        cols = [c for c in t.columns if c.name in want]  # keep original order
        if cols:
            new_tables.append(Table(name=t.name, columns=cols))

    kept_table_names = {t.name for t in new_tables}
    kept_col_pairs = {(t.name, c.name) for t in new_tables for c in t.columns}

    new_fks = [
        fk for fk in schema.foreign_keys
        if fk.src_table in kept_table_names and fk.dst_table in kept_table_names
        and (fk.src_table, fk.src_col) in kept_col_pairs
        and (fk.dst_table, fk.dst_col) in kept_col_pairs
    ]

    return DBSchema(db_id=schema.db_id, tables=new_tables, foreign_keys=new_fks)


def format_schema_min(schema: DBSchema) -> str:
    """
    Minimal pretty-printer to original style.
    """
    out = []
    if schema.db_id:
        out.append(f"【DB_ID】 {schema.db_id}")
    out.append("【Schema】")
    for t in schema.tables:
        out.append(f"# Table: {t.name}")
        out.append("[")
        for c in t.columns:
            typ = c.col_type or ""
            ex = f", Examples: [{', '.join(c.examples)}]" if c.examples else ""
            out.append(f"({c.name}:{typ}{ex})")
        out.append("]")
    if schema.foreign_keys:
        out.append("【Foreign keys】")
        for fk in schema.foreign_keys:
            out.append(f"{fk.src_table}.{fk.src_col}={fk.dst_table}.{fk.dst_col}")
    return "\n".join(out)

In [80]:
pruned = prune_schema_min(schema_obj, kept_cols)
print(format_schema_min(pruned))

【DB_ID】 california_schools
【Schema】
# Table: schools
[
(MailStrAbr:TEXT, Examples: [313 West Winton Ave.])
(MailZip:TEXT, Examples: [94544-1136, 94560-5359, 94612])
(OpenDate:DATE, Examples: [2005-08-29])
(ClosedDate:DATE, Examples: [2015-07-31])
(SOC:TEXT, Examples: [65, 66, 60])
(EdOpsCode:TEXT, Examples: [TRAD, JUV, COMM])
(Virtual:TEXT, Examples: [P, N, F])
(AdmFName1:TEXT, Examples: [L Karen, Laura, Clifford])
(AdmLName1:TEXT, Examples: [Monroe, Robell, Thompson])
(AdmLName2:TEXT, Examples: [Tsang, Koelling, Moore])
]
# Table: frpm
[
(District Code:INTEGER, Examples: [10017, 31609, 31617])
(School Name:TEXT, Examples: [FAME Public Charter])
(Educational Option Type:TEXT, Examples: [Traditional])
(Charter School (Y/N):INTEGER, Examples: [1, 0])
(Free Meal Count (K-12):REAL, Examples: [565.0, 186.0, 134.0])
]
